In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import datetime

## Rosbank ML Competition

Ссылка на соревнование: https://boosters.pro/champ_15

Росбанк – часть ведущей международной финансовой группы Societe Generale, банк включен ЦБ РФ в число 11 системно значимых кредитных организаций России. Инновации неотъемлемый процесс работы Росбанка, поэтому активно развивается направленный анализа больших данных.

- Данные

Датасет, который содержит историю транзакций клиентов за 3 месяца льготного использования банковского продукта

- Задача

Задача бинарной классификации – прогноз оттока клиентов

Колонка cl_id содержит вутренний id клиента. Для каждого уникальнго cl_id следует предсказать продолжит ли клиент пользоваться продуктом (target_flag). Значение 0 соответствует отказу, а значение 1 соответствует продолжению использования

In [102]:
raw_df = pd.read_csv('rosbank_train.csv')
raw_df.head(5)
# target_sum - можно выкинуть, переменная участвует в другой задаче

,PERIOD,cl_id,MCC,channel_type,currency,TRDATETIME,amount,trx_category,target_flag,target_sum
0,01/10/2017,0,5200,NaN,810,21OCT17:00:00:00,5023.0,POS,0,0.0
1,01/10/2017,0,6011,NaN,810,12OCT17:12:24:07,20000.0,DEPOSIT,0,0.0
2,01/12/2017,0,5921,NaN,810,05DEC17:00:00:00,767.0,POS,0,0.0
3,01/10/2017,0,5411,NaN,810,21OCT17:00:00:00,2031.0,POS,0,0.0
4,01/10/2017,0,6012,NaN,810,24OCT17:13:14:24,36562.0,C2C_OUT,0,0.0


In [32]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 490513 entries, 0 to 490512
Data columns (total 10 columns):
PERIOD          490513 non-null object
cl_id           490513 non-null int64
MCC             490513 non-null int64
channel_type    487603 non-null object
currency        490513 non-null int64
TRDATETIME      490513 non-null object
amount          490513 non-null float64
trx_category    490513 non-null object
target_flag     490513 non-null int64
target_sum      490513 non-null float64
dtypes: float64(2), int64(4), object(4)
memory usage: 37.4+ MB


In [33]:
raw_df= raw_df.drop("target_sum", axis=1)

In [34]:
raw_df.head(5)

,PERIOD,cl_id,MCC,channel_type,currency,TRDATETIME,amount,trx_category,target_flag
0,01/10/2017,0,5200,NaN,810,21OCT17:00:00:00,5023.0,POS,0
1,01/10/2017,0,6011,NaN,810,12OCT17:12:24:07,20000.0,DEPOSIT,0
2,01/12/2017,0,5921,NaN,810,05DEC17:00:00:00,767.0,POS,0
3,01/10/2017,0,5411,NaN,810,21OCT17:00:00:00,2031.0,POS,0
4,01/10/2017,0,6012,NaN,810,24OCT17:13:14:24,36562.0,C2C_OUT,0


In [35]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 490513 entries, 0 to 490512
Data columns (total 9 columns):
PERIOD          490513 non-null object
cl_id           490513 non-null int64
MCC             490513 non-null int64
channel_type    487603 non-null object
currency        490513 non-null int64
TRDATETIME      490513 non-null object
amount          490513 non-null float64
trx_category    490513 non-null object
target_flag     490513 non-null int64
dtypes: float64(1), int64(4), object(4)
memory usage: 33.7+ MB


In [36]:
print("Total clients: ", len(raw_df.cl_id.unique()))

Total clients:  5000


In [37]:
raw_df.shape

(490513, 9)

In [38]:
raw_df.cl_id.value_counts().head()

2143    784
5373    512
5630    501
4564    499
1261    485
Name: cl_id, dtype: int64

Всего 5000 клиентов, случайным образом возьмем 1000 клиентов для тестирования

In [39]:
cl_ids_test = np.random.choice(raw_df.cl_id.unique(), size=1000, replace=False)
cl_ids_test_set = set(cl_ids_test)
cl_ids_test_set 

{1,
 8195,
 2052,
 2055,
 2056,
 10,
 6155,
 2062,
 6159,
 2065,
 4116,
 20,
 8215,
 2072,
 25,
 2073,
 29,
 2078,
 2080,
 6178,
 35,
 4132,
 8230,
 2087,
 4135,
 8234,
 8235,
 4140,
 8236,
 8237,
 8239,
 2096,
 2102,
 54,
 8246,
 6200,
 4154,
 4159,
 2112,
 8258,
 2115,
 6212,
 69,
 68,
 6213,
 2116,
 2121,
 2119,
 2123,
 4172,
 81,
 8274,
 2131,
 4180,
 85,
 4182,
 2135,
 6227,
 6239,
 6240,
 2145,
 8290,
 8288,
 8292,
 8291,
 6244,
 4200,
 6250,
 6251,
 4204,
 4205,
 108,
 2157,
 4208,
 8305,
 119,
 2168,
 6272,
 6273,
 4227,
 6278,
 4231,
 136,
 134,
 138,
 139,
 8330,
 4237,
 8338,
 2196,
 8342,
 8344,
 153,
 152,
 2201,
 8352,
 2210,
 163,
 8355,
 166,
 168,
 6314,
 6315,
 8366,
 2223,
 6321,
 6322,
 8371,
 6324,
 2234,
 8378,
 4285,
 2238,
 2241,
 8388,
 6341,
 8396,
 2258,
 4307,
 4308,
 2261,
 4310,
 4306,
 8409,
 4313,
 8411,
 4316,
 224,
 225,
 8419,
 229,
 2278,
 4327,
 2282,
 2285,
 239,
 4340,
 4348,
 254,
 6399,
 6398,
 2302,
 4355,
 2308,
 6405,
 2312,
 8456,
 265,
 267

In [40]:
# create transactions dataset for train
transactions_train = raw_df[~raw_df.cl_id.isin(cl_ids_test)].copy()
print("Total transactions in train dataset: ", len(transactions_train))
# create transactions dataset for test
transactions_test = raw_df[raw_df.cl_id.isin(cl_ids_test)].copy()
print("Total transactions in test dataset: ", len(transactions_test))

Total transactions in train dataset:  389436
Total transactions in test dataset:  101077


## Домашняя работа

1. Наборы данных вида Transactions (несколько транзакций на одного клиента) трансформировать в таблицу, где cl_id будут уникальными (соответственно 4000 строк в train и 1000 строк в test
2. Для каждого cl_id будет уникальное целевое событие target_flag, а также уникальный канал привлечения клиента channel_type (клиент привлекается лишь однажды и с самого начала его записи присваивается значение канала привлечения)
3. При агрегации (*pandas.DataFrame.groupby*) по cl_id (или по связке cl_id, channel_type, target_flag) необходимо создавать производные фичи, идеи для таких фичей могут быть следующими:

    - общая сумма транзакций по каждой из trx_category
    - общая сумма транзакции по основным вылютам (напр. выделить рубли, доллары и евро - предположительно, это будут самые крупные категории)
    - общая сумма транзакций по категориям MCC кодов (например, выбрать основные/популярные MCC коды). ВНИМАНИ! Некоторые MCC коды из train могут быть не представлены в test. Про MCC коды в целом: http://www.banki.ru/wikibank/mcc-kod/; Справочник MCC кодов: https://mcc-codes.ru/code; Про некоторые категории кэшбека Росбанка: https://mcc-codes.ru/card/rosbank-sverkh-plus;
    - возможные агрегации по времени суток и дням недели - траты в выходные (праздники) или будни, в ночное время или в рабочее и т.д.
4. **Обязательная часть**: провести первичный анализ данных - посмотреть распределения признаков, выделить самые популярные MCC, помотреть активность клиентов по дням недели/времени, какие категории транзакции (trx_category) наиболее популярны и т.д. Получить инсайты, которые в дальнейшем помогут вам правильно подготовить фичи
5. **Дополнительная часть**: с отобранными фичами и полученными компонентами обучить модель (тип алгоритма на свой вкус, можно начать с линейной) и померить качество на локальном тестовом наборе данных (локальная валидация), который создается в этом ноутбуке. **Метрика оценки качества - ROC AUC**(https://en.wikipedia.org/wiki/Receiver_operating_characteristic)
6. Задания принимаются в виде ноутбука с кодов/картинками выполненной обязательной части + указанием места в leaderboard при решении дополнительной

При возникновении вопросов и для отправки домашнего задания - egsachko@gmail.com или http://fb.com/sachkoe
    

In [103]:
# выделить самые популярные MCC -вообще первый 20 типов более 80 процентов операции охватвают,
#для модели можно будет это свернуть в хвост 
raw_df['MCC'].value_counts(normalize=True).head(20)

5411    0.247985
6011    0.110868
5814    0.084302
5812    0.061216
5499    0.055528
5541    0.040399
5912    0.038180
5999    0.026652
6012    0.020501
5921    0.017488
5331    0.015578
4121    0.012774
5211    0.012766
4829    0.012650
5691    0.010522
5261    0.009792
4111    0.008893
5977    0.008766
5200    0.007054
5732    0.006669
Name: MCC, dtype: float64

In [41]:
raw_df.head()

,PERIOD,cl_id,MCC,channel_type,currency,TRDATETIME,amount,trx_category,target_flag
0,01/10/2017,0,5200,NaN,810,21OCT17:00:00:00,5023.0,POS,0
1,01/10/2017,0,6011,NaN,810,12OCT17:12:24:07,20000.0,DEPOSIT,0
2,01/12/2017,0,5921,NaN,810,05DEC17:00:00:00,767.0,POS,0
3,01/10/2017,0,5411,NaN,810,21OCT17:00:00:00,2031.0,POS,0
4,01/10/2017,0,6012,NaN,810,24OCT17:13:14:24,36562.0,C2C_OUT,0


In [43]:
#Это и попробуем сделать

In [44]:
c=raw_df['MCC'].value_counts().head(20).index.tolist()
def b(code):
    #c=raw_df['MCC'].value_counts().head(20).index.tolist()
    if code in c:
        return code
    else:
        return 'other code'
raw_df['MCC_simp'] = raw_df.apply(lambda row: b(row.MCC), axis=1)


In [45]:
raw_df['MCC_simp'] = raw_df.apply(lambda row: b(row.MCC), axis=1)


In [46]:
raw_df.trx_category.value_counts()

POS               416425
DEPOSIT            21216
WD_ATM_ROS         19104
WD_ATM_PARTNER      9948
C2C_IN              7306
WD_ATM_OTHER        7140
C2C_OUT             5456
BACK_TRX            2687
CAT                 1197
CASH_ADV              34
Name: trx_category, dtype: int64

In [47]:
#помотреть активность клиентов по дням недели/времени 	12OCT17:12:24:07

In [48]:
from datetime import datetime
import calendar



In [49]:
# не знаю зачем создадим названия дней недели
raw_df['day_of_week'] = raw_df['TRDATETIME'].apply(lambda x: calendar.day_name[datetime.strptime(x, '%d%b%y:%H:%M:%S').weekday()])

In [50]:
#И номера (0 -понедельник)
raw_df['day_of_week_number'] = raw_df['TRDATETIME'].apply(lambda x: datetime.strptime(x, '%d%b%y:%H:%M:%S').weekday())

In [51]:
# create "is weekend?" feature
def process_day(day):
    if day <= 4:
        return 0
    else:
        return 1

# добавим булеву перменную для выходных
raw_df['weekend'] = raw_df.apply(lambda row: process_day(row.day_of_week_number), axis=1)

In [52]:
#Создадим переменную соответвующую часам
raw_df['hour'] =  raw_df['TRDATETIME'].apply(lambda x: datetime.strptime(x, '%d%b%y:%H:%M:%S').hour)

In [53]:
#создадим перменную части дня
def process_hour(hour):
    if 6 <= hour < 12:
        return 0
    elif 12 <= hour < 18:
        return 1
    elif 18 <= hour < 24:
        return 2
    elif 0 <= hour < 6:
        return 3
raw_df['part_of_day'] = raw_df.apply(lambda row: process_hour(row.hour), axis=1)

In [54]:
#Удалим все лишние колнки
raw_df=raw_df.drop(['PERIOD','MCC','TRDATETIME'], axis=1)


In [55]:
raw_df.head()

,cl_id,channel_type,currency,amount,trx_category,target_flag,MCC_simp,day_of_week,day_of_week_number,weekend,hour,part_of_day
0,0,NaN,810,5023.0,POS,0,5200,Saturday,5,1,0,3
1,0,NaN,810,20000.0,DEPOSIT,0,6011,Thursday,3,0,12,1
2,0,NaN,810,767.0,POS,0,5921,Tuesday,1,0,0,3
3,0,NaN,810,2031.0,POS,0,5411,Saturday,5,1,0,3
4,0,NaN,810,36562.0,C2C_OUT,0,6012,Tuesday,1,0,13,1


In [56]:
#Таже картинка по валютам-98,5 % операции приходятся на 2 вадлюты, давайте укрупним
raw_df['currency'].value_counts(normalize=True).head()

810    0.973764
978    0.011286
840    0.003916
933    0.001144
985    0.000862
Name: currency, dtype: float64

In [57]:
dy=raw_df['currency'].value_counts().head(2).index.tolist()
def d(code):
    if code in dy:
        return code
    else:
        return 'other currency'
raw_df['currency_simp'] = raw_df.apply(lambda row: d(row.currency), axis=1)


In [58]:
raw_df['currency_simp'].value_counts(normalize=True).head()

810               0.973764
other currency    0.014950
978               0.011286
Name: currency_simp, dtype: float64

In [59]:
dy=raw_df['currency'].value_counts().head(2).index.tolist()
dy

[810, 978]

In [60]:
raw_df.head(20)

,cl_id,channel_type,currency,amount,trx_category,target_flag,MCC_simp,day_of_week,day_of_week_number,weekend,hour,part_of_day,currency_simp
0,0,NaN,810,5023.00,POS,0,5200,Saturday,5,1,0,3,810
1,0,NaN,810,20000.00,DEPOSIT,0,6011,Thursday,3,0,12,1,810
2,0,NaN,810,767.00,POS,0,5921,Tuesday,1,0,0,3,810
3,0,NaN,810,2031.00,POS,0,5411,Saturday,5,1,0,3,810
4,0,NaN,810,36562.00,C2C_OUT,0,6012,Tuesday,1,0,13,1,810
5,1,NaN,810,380.00,POS,0,5814,Monday,0,0,0,3,810
6,1,NaN,810,378.00,POS,0,5814,Tuesday,1,0,0,3,810
7,1,NaN,810,199.00,POS,0,5814,Monday,0,0,0,3,810
8,1,NaN,810,400.00,POS,0,5814,Wednesday,2,0,0,3,810
9,1,NaN,810,598.00,POS,0,5411,Wednesday,2,0,0,3,810


In [61]:
#Попробуем собрать наш датасет

In [88]:


def combine(dataset):
    name_of_column=['day_of_week_number','trx_category','part_of_day','currency_simp','weekend']
    x=dataset[['cl_id','target_flag','channel_type']].drop_duplicates().fillna(0)
    for name_column in name_of_column:
        t=dataset.groupby(['cl_id', name_column ])['amount'].sum().unstack().fillna(0).reset_index()
        
        x=x.merge(t, on='cl_id', how='left')
    
    
    return x

In [94]:
aggr_df=combine(raw_df)

In [98]:
#сформируем трейин и тест
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(aggr_df, test_size=0.2, random_state=42)

In [99]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4000 entries, 4227 to 860
Data columns (total 29 columns):
cl_id             4000 non-null int64
target_flag       4000 non-null int64
channel_type      4000 non-null object
0_x               4000 non-null float64
1_x               4000 non-null float64
2_x               4000 non-null float64
3_x               4000 non-null float64
4                 4000 non-null float64
5                 4000 non-null float64
6                 4000 non-null float64
BACK_TRX          4000 non-null float64
C2C_IN            4000 non-null float64
C2C_OUT           4000 non-null float64
CASH_ADV          4000 non-null float64
CAT               4000 non-null float64
DEPOSIT           4000 non-null float64
POS               4000 non-null float64
WD_ATM_OTHER      4000 non-null float64
WD_ATM_PARTNER    4000 non-null float64
WD_ATM_ROS        4000 non-null float64
0_y               4000 non-null float64
1_y               4000 non-null float64
2_y             